In [1]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
from collections import Counter
from itertools import chain
import pandas as pd
import numpy as np
import nltk
from ipywidgets import Output
from nltk.tokenize import RegexpTokenizer, TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import LdaModel
from gensim.models import Phrases
from itertools import chain
from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import networkx as nx
from snapy import MinHash, LSH

In [3]:
!pip install snapy mmh3

     |████████████████████████████████| 50 kB 1.4 MB/s eta 0:00:01


In [5]:
train = pd.read_csv('../data/external/kaggle/train.csv')
test = pd.read_csv('../data/external/kaggle/test.csv')
X = train['text'].values
y = pd.read_csv('../data/external/kaggle/train.csv')['target'].values
tokenizer = TweetTokenizer()

In [6]:
%%time
# content_exploded = [' '.join(list(item)) for item in content]
content = X.tolist()
labels = list(range(len(content)))
seed = 42
minhash = MinHash(content, n_gram=3, permutations=250, hash_bits=64, seed=seed)
lsh = LSH(minhash, labels, no_of_bands=50)
adjacency_list = lsh.adjacency_list(min_jaccard=0.8)
adjacency_list = {k:v for k,v in adjacency_list.items() if v}
# for k,v in adjacency_list.items():
#     print(f'{content[k]} is probably similar to: {[content[item] for item in v]}')

CPU times: user 2min, sys: 145 ms, total: 2min 1s
Wall time: 2min 1s


In [11]:
adjacency_list

{40: [48],
 48: [40],
 106: [115],
 115: [106],
 118: [119],
 119: [118],
 147: [164],
 164: [147],
 346: [364, 370, 372],
 347: [351, 378],
 350: [371, 373],
 351: [347, 378],
 364: [346, 370],
 370: [346, 364, 372],
 371: [350, 373],
 372: [346, 370],
 373: [371, 350],
 378: [347, 351],
 610: [624, 630, 634],
 624: [610, 630, 634],
 630: [610, 624, 634],
 634: [610, 624, 630],
 1134: [1156],
 1156: [1134],
 1172: [1623],
 1197: [1331],
 1199: [1343],
 1201: [1356],
 1202: [1335],
 1204: [1332],
 1213: [1360],
 1214: [1365],
 1221: [1349],
 1222: [1345],
 1242: [1251],
 1251: [1242],
 1331: [1197],
 1332: [1204],
 1335: [1202],
 1343: [1199],
 1345: [1222],
 1349: [1221],
 1356: [1201],
 1360: [1213],
 1365: [1214],
 1623: [1172],
 1703: [7611],
 1704: [1725],
 1725: [1704],
 1771: [4601],
 2345: [2352],
 2352: [2345],
 2439: [2441, 2449, 2454, 2477],
 2441: [2439, 2449, 2454, 2477],
 2449: [2439, 2441, 2454, 2477],
 2450: [2452, 2456],
 2452: [2450, 2456],
 2454: [2439, 2441, 2449, 2

In [17]:
content[2830]

'.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4'

In [18]:
content[2831]

'.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4'

In [19]:
content[2832]

'.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4'

In [20]:
content[2833]

'.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4'

In [22]:
adjacency_list[346]

[364, 370, 372]

In [23]:
content[346]

'Beyonce Is my pick for http://t.co/nnMQlz91o9 Fan Army #Beyhive http://t.co/o91f3cYy0R 77'

In [24]:
content[364]

'Beyonce Is my pick for http://t.co/nnMQlz91o9 Fan Army #Beyhive http://t.co/o91f3cYy0R 78'

In [25]:
content[370]

'Beyonce Is my pick for http://t.co/nnMQlz91o9 Fan Army #Beyhive http://t.co/o91f3cYy0R 72'

In [26]:
content[372]

'Beyonce Is my pick for http://t.co/nnMQlz91o9 Fan Army #Beyhive http://t.co/o91f3cYy0R 66'

In [39]:
duplicate_graph = nx.from_dict_of_lists(adjacency_list)

In [40]:
for node, attr in duplicate_graph.nodes(data=True):
    attr['tweet'] = content[node]
    attr['label'] = 'disaster' if train['target'][node] else 'non-disaster'

In [41]:
duplicate_graph.nodes.get(40)

{'tweet': 'Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw',
 'label': 'non-disaster'}

In [42]:
nx.write_graphml(duplicate_graph, '../data/processed/duplicates.graphml')

In [23]:
train['target'][40]

0

In [21]:
content[40]

'Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw'